In [1]:
import torch
import torch.nn as nn

import numpy as np
import pickle as pk

from scipy.sparse import csr_matrix, lil_matrix
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = []
for d in range(15, 22):
    filename = '/data/fan/UsersInOsakaProcessed/201210{:02d}_interp.pk'.format(d)
    print(filename)
    with open(filename, 'rb') as f:
        data += list(pk.load(f).values())
data = np.stack(data, axis=0)

/data/fan/UsersInOsakaProcessed/20121015_interp.pk
/data/fan/UsersInOsakaProcessed/20121016_interp.pk
/data/fan/UsersInOsakaProcessed/20121017_interp.pk
/data/fan/UsersInOsakaProcessed/20121018_interp.pk
/data/fan/UsersInOsakaProcessed/20121019_interp.pk
/data/fan/UsersInOsakaProcessed/20121020_interp.pk
/data/fan/UsersInOsakaProcessed/20121021_interp.pk


In [3]:
val_data = []
for d in range(22, 29):
    filename = '/data/fan/UsersInOsakaProcessed/201210{:02d}_interp.pk'.format(d)
    print(filename)
    with open(filename, 'rb') as f:
        val_data += list(pk.load(f).values())
val_data = np.stack(val_data, axis=0)

/data/fan/UsersInOsakaProcessed/20121022_interp.pk
/data/fan/UsersInOsakaProcessed/20121023_interp.pk
/data/fan/UsersInOsakaProcessed/20121024_interp.pk
/data/fan/UsersInOsakaProcessed/20121025_interp.pk
/data/fan/UsersInOsakaProcessed/20121026_interp.pk
/data/fan/UsersInOsakaProcessed/20121027_interp.pk
/data/fan/UsersInOsakaProcessed/20121028_interp.pk


In [4]:
data.shape

(250770, 96)

In [17]:
global_predictor = RandomForestClassifier(n_estimators=50, max_depth=4)

In [15]:
T = 96
dT = 4
num_clusters = 1600

In [16]:
data_x = np.zeros((data.shape[0], dT * num_clusters + T))
data_y = []
for i in range(data.shape[0]):
    t = np.random.randint(T - 2 * dT)
    data[i, t: t + dT]
    data_x[i, t] = 1.
    data_x[i, T + data[i, t]] = 1.
    data_x[i, T + num_clusters + data[i, t + 1]] = 1.
    data_x[i, T + 2 * num_clusters + data[i, t + 2]] = 1.
    data_x[i, T + 3 * num_clusters + data[i, t + 3]] = 1.
    data_y.append(data[i, t + 2 * dT - 1])

In [18]:
global_predictor.fit(data_x, data_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
with open('./results_osaka/global_predictor_randomforest.pk', 'wb') as f:
    pk.dump(global_predictor, f)